# Маршрутизация в Linux

### Геогрий Курячий

## Интерфейсный уровень

#### https://www.youtube.com/watch?v=E_gA-DehDi4&list=PL6kSdcHYB3x7IaYsg-MaN6vuxq7YVKK7D

Роутер - инструмент для обработки, перенаправления и фильтрации сетевого трафика.

Элементы сети подготавливаются в VirtualBox. Гостевая ОС - ALT Рабочая станция 9.1.1 с графическим интерфейсом

### Сервер 

```bash
user ~ su -
```

#### Характеристики:

- Общие
    - Имя: Сервер
    - ОС: Limux 2.6 / 3.x / 4.x (64-bit)
    - Лекция про сеть в Linux
- Система
    - Оперативная память: 1024 МБ
    - Порядок загрузки: Гибкий диск, Оптический диск, Жесткий диск
    - Ускорение: VT-x/AMD-V, Nested Paging, Паравиртуализация KVM
- Дисплей
    - Видеопамять: 64 МБ
    - Графический контролер: VMSVGA
    - Ускорение: 3D-ускорение
    - Сервер удаленного дисплея: Выключен
    - Запись: Выключена
- Носители
    - Контроллер: IDE
        - Вторичный мастер IDE: \[Оптический привод\] Пусто
    - Контроллер SATA
        - SATA порт 0: Alt.vdi (Обычный, 32,00 ГБ)
- Аудио
    - Аудиодравйвер: PulseAudio
    - Аудиоконтроллер: ICH AC97
- Сеть
    - Адаптер 1: Intel PRO/1000 MT Desktop (NAT)
    - Адаптер 2: Паравиртуальная сеть (Внутренняя сеть, 'isolated')
- COM-порты
    - Выключены
- USB
    - USB-контроллер: OHCI

#### Сетевые интерфейсы

Содержит два сетевых интефейса:
1. Смотрит наружу: NAT (в терминах VirtualBox) - имеет соединение с Интернетом
2. Смотрит внутрь: isolated (служит для привязки других машин) - фактически является сетевым кабелем, вставленным в аппаратуру и с помощью которого можно подключить другие машины к серверу

Проверка сетевых интерфейсов:
```bash
srv ~ ip a
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc no queue state UNKNOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
        valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
        valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global dynamic noprefixroute eth0
        valid_lft 86346sec preferred_lft 75546sec
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
    inet 10.20.30.1/24 brd 10.20.30.255 scope global eth1
        valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fed1:fd2b/64 scope link
        valid_lft forever preferred_lft forever
```

Соответствие сетевых интерфейсов:
1. eth0 (NAT): 10.0.2.15 - типичный внешний адрес для гистов VirtualBox
2. eth1 (isolated): 10.20.30.1 - самостоятельно настроенный адрес

#### Остановка сети

Остановка настроеной сети:
```bash
srv ~ systemctl stop network
srv ~ ip a
1: lo: <LOOPBACK> mtu 65536 qdisc noqueue state DOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00r
2: eth0: <BROADCAST,MULTICAST> mtu 1500 qdisc fq_codel state DOWN group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
3: eth1: <BROADCAST,MULTICAST> mtu 1500 qdisc fq_codel state DOWN group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
```

Соответствующие сетевые интерфейсы сохранились, но не настроены

#### Настройка внешнего интерфейса сети

Настройка внешнего интерфейса с помощью демона, использующего внешние настройки от провайдера:
```bash
srv ~ dhcpcd eth0
eth0: waiting for carrier
eth0: carrier acquired
eth0: rebinding lease of 10.0.2.15
eth0: leased 10.0.2.15 for 86400 seconds
eth0: adding route to 10.0.2.0/24
eth0: adding default route via 10.0.2.2
forked to background, child pid 4017
srv ~ ip a
1: lo: <LOOPBACK> mtu 65536 qdisc noqueue state DOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global dynamic noprefixroute eth0
        valid_lft 86398sec preferred_lft 75598sec
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST> mtu 1500 qdisc fq_codel state DOWN group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
```

Демон ходит к провайдеру и всё поднимает (в том числе и интерфейс)

#### Настройка внутреннего интерфейса сети

Для соединения клиента с сервером нужно настроить внутренний (внешний для клиента) интерфейс сервера (дать ему адрес) и включить демон настройки.

Поднятие интерфейса:
```bash
srv ~ ip link set eth1 up
srv ~ ip a
1: lo: <LOOPBACK> mtu 65536 qdisc noqueue state DOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global dynamic noprefixroute eth0
        valid_lft 86398sec preferred_lft 75598sec
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
```

Присваивание адреса интерфейса:
```bash
srv ~ ip addr add 10.20.30.1/24 dev eth1
srv ~ ip a
1: lo: <LOOPBACK> mtu 65536 qdisc noqueue state DOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global dynamic noprefixroute eth0
        valid_lft 86398sec preferred_lft 75598sec
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
        inet 10.0.2.15/24 scope global eth1
        valid_lft 86398sec preferred_lft 75598sec
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
```

Число 24 в конце адреса означает количество единиц в сетевой маске, растущей слева направо (то есть 24 эквивалентно 255.255.255.255) - 24 бита единиц слева от адреса

Проверка запуска dhcpd:
```bash
srv ~ ps gaxwww | grep dhpcd
    2896 ?        Ss     0:00 /usr/sbin/dhcpd -4 -f --no-pid
    4036 pts/0    S+     0:00 grep --color=auto dhcpd
```

Подслушивание трафика внутреннего интерфейса:
```bash
srv ~ tcpdump -i eth1
```

#### Настройка внешнего интерфейса сети

Физически подключенные к настроенному интерфейсу клиенты сразу получают доступ к нему доступ, однако не получают доступ к внешнему интерфейсу сервера. Для этого необходимо перекинуть адрес внутреннего (внешнего для клиента) интерфейса (10.20.30.1/24) на внешний (10.0.2.15/24). Помимо этого, пакеты с адресами 10/8 не должны существовать во внешнем интернете и не будут туда маршрутизированы, из-за чего ответ на этот запрос (пинг) не вернется.

Для получения доступа к внешнему интерфейсу необходимо включить трансляцию адресов таким образом, чтобы при выходе из внешнего интерфейса пакет с адресом 10.20.30.1 превращался в пакет с адресом 10.0.2.15

Включение трансляции адресов:
```bash
srv ~ iptables -t nat -o eth0 -A POSTROUTING -j MASQUERADE
```

В результате во всем трафик при выходе из интерфейса eth0 в самый последний момент (после того как маршрутизация определена) с помощью трансляции адресов подменяется адрес с того, что был, на некоторый внешний, а при входе в этот интерфейс - наоборот.

Проверка результата включения трансляции:
```bash
srv ~ iptables-save
# Generated by iptables-save v1.0.3 on Tue Mar 30 14:18:15 2021 
*nat
:PREROUTING ACCEPT [4:268]
:INPUT ACCEPT [0:0]
:OUTPUT ACCEPT [0:0]
:POSTROUTING ACCEPT [0:0]
-A POSTROUTING -o eth0 -j MASQUERADE
COMMIT
# Completed on Tue Mar 3 14:18:15 2021
```

Теперь запрос (пинг) клиента на внешний адрес можно увидеть в трафике внешнего интерфейса вместе с трансляцией адресов, однако трафик внутреннего интерфейса сохранится без трансляции, что также можно увидеть.

Подслушивание трафика внешнего интерфейса вместе с информацией о трансляции адресов:
```bash
srv ~ tcpdump -ni eth0
```

#### Проверка производимой сервером маршрутизации

Вывод статической таблицы маршрутизации:
```bash
srv ~ ip poute
default via 10.0.2.2 dev eth0 proto dhcp src 10.0.2.15 metric 202
10.0.2.0/24 dev eth0 proto dhcp scope link src 10.0.2.15 metric 202
10.20.30.0/24 dev eth1 proto kernel scope link src 10.20.30.1
```

В данном примере имеется три записи:
1. Направление по умолчанию (с сетевой маской 0)
2. Локальная сеть за интерфейсом eth0
3. Локальная сеть за интерфейсом eth1

Далее применяются стандартные сетевые правила:
1. Чем больше сетевая маска (единицы после слеша), тем точнее перенаправление и приоритетнее маршрут в таблице (и наоборот)

Таким образом, теперь сервер маршрутизирует все, что лежит за интерфейсами eth0 и eth1: 
1. Пакет от 10.20.30.10 перекладывается в 10.0.2.0 по умолчанию
2. Пакет обрабатывается межсетевым экраном для произведения трансляции адресов, иначе пакет не вернется обратно

### Простой клиент

```bash
user ~ su -
```

#### Характеристики:

- Система
    - Оперативная память: 1024 МБ
    - Порядок загрузки: Гибкий диск, Оптический диск, Жесткий диск
    - Ускорение: VT-x/AMD-V, Nested Paging, Паравиртуализация KVM
- Дисплей
    - Видеопамять: 64 МБ
    - Графический контролер: VMSVGA
    - Ускорение: 3D-ускорение
    - Сервер удаленного дисплея: Выключен
    - Запись: Выключена
- Носители
    - Контроллер: IDE
        - Вторичный мастер IDE: \[Оптический привод\] Пусто
    - Контроллер SATA
        - SATA порт 0: Alt.vdi (Обычный, 32,00 ГБ)
- Аудио
    - Аудиодравйвер: PulseAudio
    - Аудиоконтроллер: ICH AC97
- Сеть
    - Адаптер 1: Intel PRO/1000 MT Desktop (Внутренняя сеть, 'isolated')
- COM-порты
    - Выключены
- USB
    - USB-контроллер: OHCI
    - Фильтры устройств: 0 (0 активно)

#### Сетевые интерфейсы

Подключение к isolated будет невозможно, пока сервер не запустит это соединение. Доступ к внешним ресурсам будет невозможен, пока сервер не настроит трансляцию адресов. 

Также необходимо настроить nameserver на клиенте для преобразования именных адресов в IP-адреса.

#### Проверка настройки nameserver

```bash
srv ~ cat /etc/resolv.conf
# Generater by resolvconf
# Do not edit manually, use
# /etc/net/ifaces/<interface>/resolv.conf instead
nameserver 192.168.100.1
```

#### Проверка подключения к серверу при настроенном внутреннем интерфейсе сервера

```bash
srv ~ systemctl restart network
srv ~ ping 10.20.30.1
```

#### Проверка подключения к внешним ресурсам при настроенном внешнем интерфейсе сервера

```bash
srv ~ ping ww.ru
```

# NO

2. Смотрит внутрь: isolated (служит для привязки других машин) - фактически является сетевым кабелем, вставленным в аппаратуру и с помощью которого можно подключить другие машины к серверу

Проверка сетевых интерфейсов:
```bash
user# su -
srv# ip a
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc no queue state UNKNOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
        valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
        valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global dynamic noprefixroute eth0
        valid_lft 86346sec preferred_lft 75546sec
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
    inet 10.20.30.1/24 brd 10.20.30.255 scope global eth1
        valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fed1:fd2b/64 scope link
        valid_lft forever preferred_lft forever
```

Соответствие сетевых интерфейсов:
1. eth0 (NAT): 10.0.2.15 - типичный внешний адрес для гистов VirtualBox
2. eth1 (isolated): 10.20.30.1 - самостоятельно настроенный адрес

#### Остановка сети

Остановка настроеной сети:
```bash
srv# systemctl stop network
srv# ip a
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc no queue state UNKNOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
        valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
        valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state DOWN group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state DOWN group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
```

Соответствующие сетевые интерфейсы сохранились, но не настроены



#### Характеристики:

- Общие
    - Имя: Сервер
    - ОС: Limux 2.6 / 3.x / 4.x (64-bit)
    - Лекция про сеть в Linux
- Система
    - Оперативная память: 1024 МБ
    - Порядок загрузки: Гибкий диск, Оптический диск, Жесткий диск
    - Ускорение: VT-x/AMD-V, Nested Paging, Паравиртуализация KVM
- Дисплей
    - Видеопамять: 64 МБ
    - Графический контролер: VMSVGA
    - Ускорение: 3D-ускорение
    - Сервер удаленного дисплея: Выключен
    - Запись: Выключена
- Носители
    - Контроллер: IDE
        - Вторичный мастер IDE: \[Оптический привод\] Пусто
    - Контроллер SATA
        - SATA порт 0: Alt.vdi (Обычный, 32,00 ГБ)
- Аудио
    - Аудиодравйвер: PulseAudio
    - Аудиоконтроллер: ICH AC97
- Сеть
    - Адаптер 1: Intel PRO/1000 MT Desktop (NAT)
    - Адаптер 2: Паравиртуальная сеть (Внутренняя сеть, 'isolated')
- COM-порты
    - Выключены
- USB
    - USB-контроллер: OHCI

#### Сетевые интерфейсы

Содержит два сетевых интефейса:
1. Смотрит наружу: NAT (в терминах VirtualBox) - имеет соединение с Интернетом
2. Смотрит внутрь: isolated (служит для привязки других машин) - фактически является сетевым кабелем, вставленным в аппаратуру и с помощью которого можно подключить другие машины к серверу

Проверка сетевых интерфейсов:
```bash
user# su -
srv# ip a
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc no queue state UNKNOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
        valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
        valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global dynamic noprefixroute eth0
        valid_lft 86346sec preferred_lft 75546sec
    inet6 fe80::a00:27ff:fe04:6445/64 scope link
        valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
    inet 10.20.30.1/24 brd 10.20.30.255 scope global eth1
        valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fed1:fd2b/64 scope link
        valid_lft forever preferred_lft forever
```

Соответствие сетевых интерфейсов:
1. eth0 (NAT): 10.0.2.15 - типичный внешний адрес для гистов VirtualBox
2. eth1 (isolated): 10.20.30.1 - самостоятельно настроенный адрес

#### Остановка сети

Остановка настроеной сети:
```bash
srv# systemctl stop network
srv# ip a
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc no queue state UNKNOWN group default qlen 1000
    link/ loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
        valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
        valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state DOWN group default qlen 1000
    link/ether 08:00:27:04:64:45 brd ff:ff:ff:ff:ff:ff
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state DOWN group default qlen 1000
    link/ether 08:00:27:d1:fd:2b brd ff:ff:ff:ff:ff:ff
```

Соответствующие сетевые интерфейсы сохранились, но не настроены